Homework 7 - Heather Turcios

# Exercise 10.1

## Question 1

In [1]:
#Import Packages
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
import osmnx as ox
import matplotlib.pyplot as plt
import folium
from pyproj import crs
import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats
import os
%matplotlib inline

In [2]:
#Load Data
fp_schools = "data/Schools_LAUSD.geojson"
fp_parks = "data/Recreation_and_Parks.geojson"

#Read the data
data_schools = gpd.read_file(fp_schools)
data_parks = gpd.read_file(fp_parks)

In [3]:
#Checking to see if CRS is the same
print("The CRS for School data is", data_schools.crs)
print("The CRS for Parks data is", data_parks.crs)

The CRS for School data is epsg:4326
The CRS for Parks data is epsg:4326


## Question 2

In [4]:
#Making copies of original data 
data_schools_orig = data_schools.copy()
data_parks_orig = data_parks.copy()

In [5]:
#Checking Column Names to find necessary data
data_schools.head()

,ID,LOCN,MPD_NAME,ADDRESS,CITY,ZIP,MPD_TYPE,MAP_TYPE,LABEL,MAP_DESC,MPD_DESC,LD,FULLNAME,CDSCODE,CHARTER,MAP_TYPE_MAPPING,TOOLTIP,NLA_URL,geometry
0,1,2835,VISTA DEL VALLE AC DL,12411 N BROMONT AVE,LOS ANGELES,91340,DS,ED,Vista del Valle Ac DL,Spanish Dual Language Program,Dual Language Program,0,Vista del Valle Dual Language Academy Dual Lan...,,,ES,Name: Vista del Valle Dual Language Academy Du...,navigatela/reports/lausd_school_report.cfm?PK=1,POINT (-118.42169 34.29553)
1,2,2820,PARMELEE AVE EL DL SP,1338 E 76TH PL,LOS ANGELES,90001,DS,ED,Parmelee Ave El DL Sp,Spanish Dual Language Program,Dual Language Program,0,Parmelee Avenue Elementary Dual Language Spanish,,,ES,Name: Parmelee Avenue Elementary Dual Language...,navigatela/reports/lausd_school_report.cfm?PK=2,POINT (-118.25099 33.97006)
2,3,2773,OCHOA LC DLC SP,5057 LIVE OAK ST,CUDAHY,90201,DS,ED,Ochoa LC DLC Sp,Spanish Dual Language Program,Dual Language Program,0,Ellen Ochoa Learning Center Dual Language Spanish,,,ES,Name: Ellen Ochoa Learning Center Dual Languag...,navigatela/reports/lausd_school_report.cfm?PK=3,POINT (-118.17660 33.96793)
3,4,2840,WILTON PL EL DLC KO,745 S WILTON PL,LOS ANGELES,90005,DK,ED,Wilton Pl El DLC Ko,Korean Dual Language Program,Dual Language Program,0,Wilton Place Elementary Dual Language Korean,,,ES,Name: Wilton Place Elementary Dual Language Ko...,navigatela/reports/lausd_school_report.cfm?PK=4,POINT (-118.31537 34.05853)
4,5,2814,MILES AVE EL DLC SP,6020 MILES AVE,HUNTINGTON PARK,90255,DS,ED,Miles Ave El DLC Sp,Spanish Dual Language Program,Dual Language Program,0,Miles Avenue Elementary Dual Language Spanish,,,ES,Name: Miles Avenue Elementary Dual Language Sp...,navigatela/reports/lausd_school_report.cfm?PK=5,POINT (-118.21780 33.97755)


In [6]:
#Selecting only columns that are needed
schools_selected = data_schools[['geometry', 'FULLNAME', 'LOCN', 'ADDRESS']]
schools_selected.head()

,geometry,FULLNAME,LOCN,ADDRESS
0,POINT (-118.42169 34.29553),Vista del Valle Dual Language Academy Dual Lan...,2835,12411 N BROMONT AVE
1,POINT (-118.25099 33.97006),Parmelee Avenue Elementary Dual Language Spanish,2820,1338 E 76TH PL
2,POINT (-118.17660 33.96793),Ellen Ochoa Learning Center Dual Language Spanish,2773,5057 LIVE OAK ST
3,POINT (-118.31537 34.05853),Wilton Place Elementary Dual Language Korean,2840,745 S WILTON PL
4,POINT (-118.21780 33.97755),Miles Avenue Elementary Dual Language Spanish,2814,6020 MILES AVE


In [7]:
#Checking Column Names to find necessary data
data_parks.head()

,OBJECTID,Area_Acres,Park_ID,CD_Real,Address,Region,Name,Park_Class,CD_NEW,TOOLTIP,ACRES,geometry
0,1,8.433364,512,03,23751 Ingomar St,V,Lazy J Ranch Park,N,12,Name: Lazy J Ranch Park\nLocation: 23751 Ingom...,8.4333635,"POLYGON ((-118.64451 34.21238, -118.64451 34.2..."
1,2,1.387374,883,15,Adjacent to Angels Gate Park at 3400 Gaffey St...,P,Lookout Point Park,N,15,Name: Lookout Point Park\nLocation: Adjacent t...,1.38737377,"POLYGON ((-118.29192 33.71197, -118.29192 33.7..."
2,3,3.173861,320,15,1464 E 109th Street,P,109th Street Recreation Center,N,15,Name: 109th Street Recreation Center\nLocation...,3.17386107,"POLYGON ((-118.24925 33.93590, -118.24917 33.9..."
3,4,0.154871,366,08,4726 8th Avenue,P,Angeles Mesa Park,N,8,Name: Angeles Mesa Park\nLocation: 4726 8th Av...,0.1548711,"POLYGON ((-118.32523 34.00014, -118.32523 34.0..."
4,5,0.960725,213,09,4800 S. Hoover St,P,Julian C. Dixon Park,N,9,Name: Julian C. Dixon Park\nLocation: 4800 S. ...,0.96072542,"POLYGON ((-118.28680 33.99881, -118.28664 33.9..."


In [8]:
#Selecting only columns that are needed
parks_selected = data_parks[['geometry', 'Name', 'Address']]
parks_selected.head()

,geometry,Name,Address
0,"POLYGON ((-118.64451 34.21238, -118.64451 34.2...",Lazy J Ranch Park,23751 Ingomar St
1,"POLYGON ((-118.29192 33.71197, -118.29192 33.7...",Lookout Point Park,Adjacent to Angels Gate Park at 3400 Gaffey St...
2,"POLYGON ((-118.24925 33.93590, -118.24917 33.9...",109th Street Recreation Center,1464 E 109th Street
3,"POLYGON ((-118.32523 34.00014, -118.32523 34.0...",Angeles Mesa Park,4726 8th Avenue
4,"POLYGON ((-118.28680 33.99881, -118.28664 33.9...",Julian C. Dixon Park,4800 S. Hoover St


In [9]:
#Creating a centroid point for parks data
parks_selected['centroid'] = parks_selected.centroid
parks_selected.head()

/tmp/ipykernel_80975/2612591994.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parks_selected['centroid'] = parks_selected.centroid
/opt/tljh/user/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,geometry,Name,Address,centroid
0,"POLYGON ((-118.64451 34.21238, -118.64451 34.2...",Lazy J Ranch Park,23751 Ingomar St,POINT (-118.64515 34.21312)
1,"POLYGON ((-118.29192 33.71197, -118.29192 33.7...",Lookout Point Park,Adjacent to Angels Gate Park at 3400 Gaffey St...,POINT (-118.29214 33.71246)
2,"POLYGON ((-118.24925 33.93590, -118.24917 33.9...",109th Street Recreation Center,1464 E 109th Street,POINT (-118.24859 33.93632)
3,"POLYGON ((-118.32523 34.00014, -118.32523 34.0...",Angeles Mesa Park,4726 8th Avenue,POINT (-118.32545 34.00021)
4,"POLYGON ((-118.28680 33.99881, -118.28664 33.9...",Julian C. Dixon Park,4800 S. Hoover St,POINT (-118.28673 33.99919)


In [10]:
#Creating function to help do all distances at once
def get_nearest_values(row, other_gdf, point_column='geometry', value_column="geometry"):
    """Find the nearest point and return the corresponding value from specified value column."""
    
    # Create an union of the other GeoDataFrame's geometries:
    other_points = other_gdf["centroid"].unary_union
    
    # Find the nearest points
    nearest_geoms = nearest_points(row[point_column], other_points)
    
    # Get corresponding values from the other df
    nearest_data = other_gdf.loc[other_gdf["centroid"] == nearest_geoms[1]]
    
    nearest_value = nearest_data[value_column].values[0]
    
    return nearest_value

In [11]:
#Applying function to parks
schools_selected["nearest_loc"] = schools_selected.apply(get_nearest_values, 
                                                     other_gdf=parks_selected, point_column="geometry", value_column="Name", axis=1)
schools_selected.head()

/opt/tljh/user/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,geometry,FULLNAME,LOCN,ADDRESS,nearest_loc
0,POINT (-118.42169 34.29553),Vista del Valle Dual Language Academy Dual Lan...,2835,12411 N BROMONT AVE,Hubert H. Humphrey Memorial Park
1,POINT (-118.25099 33.97006),Parmelee Avenue Elementary Dual Language Spanish,2820,1338 E 76TH PL,Fremont High School Pool
2,POINT (-118.17660 33.96793),Ellen Ochoa Learning Center Dual Language Spanish,2773,5057 LIVE OAK ST,Jordan Downs Recreation Center
3,POINT (-118.31537 34.05853),Wilton Place Elementary Dual Language Korean,2840,745 S WILTON PL,Harold A. Henry Park
4,POINT (-118.21780 33.97755),Miles Avenue Elementary Dual Language Spanish,2814,6020 MILES AVE,Pueblo del Rio Recreation Center


## Question 3

In [12]:
schools_selected.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [13]:
parks_selected.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [14]:
#Change the name of a column
schools_selected = schools_selected.rename(columns={'nearest_loc': 'Nearest Park Name'})

In [15]:
#Change the name of a column
schools_selected = schools_selected.rename(columns={'FULLNAME': 'School Name'})

In [16]:
#Checking if renamed columns worked
schools_selected.head()

,geometry,School Name,LOCN,ADDRESS,Nearest Park Name
0,POINT (-118.42169 34.29553),Vista del Valle Dual Language Academy Dual Lan...,2835,12411 N BROMONT AVE,Hubert H. Humphrey Memorial Park
1,POINT (-118.25099 33.97006),Parmelee Avenue Elementary Dual Language Spanish,2820,1338 E 76TH PL,Fremont High School Pool
2,POINT (-118.17660 33.96793),Ellen Ochoa Learning Center Dual Language Spanish,2773,5057 LIVE OAK ST,Jordan Downs Recreation Center
3,POINT (-118.31537 34.05853),Wilton Place Elementary Dual Language Korean,2840,745 S WILTON PL,Harold A. Henry Park
4,POINT (-118.21780 33.97755),Miles Avenue Elementary Dual Language Spanish,2814,6020 MILES AVE,Pueblo del Rio Recreation Center


In [17]:
#Create a Map instance
map = folium.Map(location=[34.052235, -118.243683], zoom_start=10, control_scale=True)
map

In [18]:
#Convert school points to GeoJSON
schools_gjson = folium.features.GeoJson(schools_selected['geometry'], name="School Name")

In [22]:
#Create a Map instance
map = folium.Map(location=[34.052235, -118.243683], zoom_start=10, control_scale=True)

#Add points to the map instance
schools_gjson.add_to(map)

folium.features.GeoJson(schools_selected,  
                        name='School Name',
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['School Name', 'Nearest Park Name'],
                                                                aliases = ['School Name:', 'Nearest Park Name:'],
                                                                labels=True,
                                                                sticky=False
                                                                            )
                       ).add_to(map)

#Show map
map

## Question 4

In [24]:
outfp = "hw7_lausd_parks_map.html"
map.save(outfp)